# Árboles de Desición


In [7]:
import math
import random
from collections import Counter

def entrena_arbol(datos, target, clase_default, 
                  max_profundidad=None, acc_nodo=1.0, min_ejemplos=0,
                  variables_seleccionadas=None):
    """
    Entrena un árbol de decisión utilizando el criterio de entropía con selección aleatoria de atributos.
    """
    atributos = list(datos[0].keys())
    atributos.remove(target)
    
    if variables_seleccionadas is not None and isinstance(variables_seleccionadas, int):
        atributos = random.sample(atributos, min(variables_seleccionadas, len(atributos)))
    
    # Criterios para determinar si es un nodo hoja
    if len(datos) == 0 or len(atributos) == 0:
        return NodoN(terminal=True, clase_default=clase_default)
    
    clases = Counter(d[target] for d in datos)
    clase_default = clases.most_common(1)[0][0]
    
    if (max_profundidad == 0 or 
        len(datos) <= min_ejemplos or 
        clases.most_common(1)[0][1] / len(datos) >= acc_nodo):
        return NodoN(terminal=True, clase_default=clase_default)
    
    variable, valor = selecciona_variable_valor(datos, target, atributos)
    nodo = NodoN(
        terminal=False, 
        clase_default=clase_default,
        atributo=variable, 
        valor=valor 
    )
    nodo.hijo_menor = entrena_arbol(
        [d for d in datos if d[variable] < valor],
        target,
        clase_default,
        max_profundidad - 1 if max_profundidad is not None else None,
        acc_nodo, min_ejemplos, variables_seleccionadas
    )   
    nodo.hijo_mayor = entrena_arbol(
        [d for d in datos if d[variable] >= valor],
        target,
        clase_default,
        max_profundidad - 1 if max_profundidad is not None else None,
        acc_nodo, min_ejemplos, variables_seleccionadas
    )   
    return nodo


In [10]:
import random
from collections import Counter

def entrena_bosque(datos, target, clase_default, 
                   num_arboles=10, max_profundidad=None, 
                   acc_nodo=1.0, min_ejemplos=0, 
                   variables_seleccionadas=None):
    """
    Entrena un bosque aleatorio de árboles de decisión.
    
    Parámetros:
    - datos: conjunto de datos de entrenamiento
    - target: la clase o atributo objetivo
    - clase_default: la clase por defecto si no se puede hacer predicción
    - num_arboles: número de árboles en el bosque
    - max_profundidad: profundidad máxima de los árboles
    - acc_nodo: precisión mínima de un nodo para seguir dividendo
    - min_ejemplos: número mínimo de ejemplos para hacer una división
    - variables_seleccionadas: número de variables aleatorias por nodo
    
    Devuelve:
    - Una lista de árboles entrenados (el bosque)
    """
    bosque = []
    
    # Entrenamos 'num_arboles' árboles
    for _ in range(num_arboles):
        # Muestreo con reemplazo para crear un subconjunto de datos
        subconjunto = [random.choice(datos) for _ in range(len(datos))]
        
        # Entrenamos un árbol con el subconjunto de datos
        arbol = entrena_arbol(subconjunto, target, clase_default, 
                              max_profundidad, acc_nodo, 
                              min_ejemplos, variables_seleccionadas)
        
        # Añadimos el árbol al bosque
        bosque.append(arbol)
    
    return bosque

def predice_bosque(bosque, ejemplo):
    """
    Hace predicciones usando un bosque aleatorio.
    
    Parámetros:
    - bosque: lista de árboles entrenados
    - ejemplo: un diccionario con los atributos de entrada
    
    Devuelve:
    - La clase predicha para el ejemplo, basada en la votación mayoritaria
    """
    # Recolectamos las predicciones de todos los árboles
    predicciones = [predice_arbol(arbol, ejemplo) for arbol in bosque]
    
    # Votación mayoritaria: la clase más común entre las predicciones
    return Counter(predicciones).most_common(1)[0][0]

def predice_arbol(arbol, ejemplo):
    """
    Predice la clase de un ejemplo usando un árbol de decisión.
    
    Parámetros:
    - arbol: el árbol de decisión entrenado
    - ejemplo: un diccionario con los atributos de entrada
    
    Devuelve:
    - La clase predicha por el árbol
    """
    # Navegamos por el árbol hasta llegar a una hoja
    while not arbol.terminal:
        if ejemplo[arbol.atributo] < arbol.valor:
            arbol = arbol.hijo_menor
        else:
            arbol = arbol.hijo_mayor
    return arbol.clase_default


In [12]:
import csv

def cargar_datos(archivo):
    """
    Carga los datos desde un archivo CSV.
    
    Parámetros:
    - archivo: nombre del archivo CSV
    
    Devuelve:
    - Una lista de diccionarios, donde cada diccionario representa una fila de datos
    """
    datos = []
    with open(archivo, newline='', encoding='utf-8') as f:
        lector = csv.DictReader(f)
        for fila in lector:
            # Convertimos cada fila a un diccionario
            datos.append({clave: float(valor) if valor.replace('.', '', 1).isdigit() else valor 
                          for clave, valor in fila.items()})
    return datos
